<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/sglogo.png width=50 align="left">

# Vault Authentication Methods
<img src="images/vault-auth-1.png" width=600>  


Vault supports many different authentication methods, suitable for human and applications (e.g. AppRole, AWS/Azure/Google, K8s).
Most Vault auth methods need to be explicitly enabled.  Each method has a default path.


**Prerequisites:** 
You need to have Vault up and running

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`

In [1]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [2]:
vault login root

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       dRq6usE3NFp7vjYtgtH8uqQv
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [ ]:
vault status; vault version

---
# Userpass Authentication method
authenticates users with usernames and passwords managed by Vault.



### Setup

In [3]:
vault auth enable userpass

Success! Enabled userpass auth method at: userpass/


**Configure username/password**

In [4]:
vault auth enable userpass
vault write auth/userpass/users/arthur password="passw0rd" policies="static-policy"
vault write auth/userpass/users/samantha password="passw0rd" policies="static-policy"
vault write auth/userpass/users/tio password="passw0rd" policies="static-policy, dynamic-policy"
vault write auth/userpass/users/moayad password="passw0rd" policies="static-policy, dynamic-policy"
vault write auth/userpass/users/raymond password="passw0rd" policies="static-policy, dynamic-policy"

Error enabling userpass auth: Error making API request.

URL: POST http://127.0.0.1:8200/v1/sys/auth/userpass
Code: 400. Errors:

* path is already in use at userpass/
Success! Data written to: auth/userpass/users/arthur
Success! Data written to: auth/userpass/users/samantha
Success! Data written to: auth/userpass/users/tio
Success! Data written to: auth/userpass/users/moayad
Success! Data written to: auth/userpass/users/raymond


### Login

In [ ]:
unset VAULT_TOKEN
vault login -method=userpass username=tio password=passw0rd

**Via cURL**
The response will contain the token at `auth.client_token`

In [ ]:
curl -s --request POST --data '{"password": "passw0rd"}' \
    http://127.0.0.1:8200/v1/auth/userpass/login/tio | jq

In [ ]:
vault list auth/userpass/users

---
# LDAP Authentication Method
<img src="images/ldap-auth-workflow.svg" width=700>  


Vault supports many different authentication methods, suitable for human and applications (e.g. AppRole, AWS/Azure/Google, K8s).
Most Vault auth methods need to be explicitly enabled.  Each method has a default path.


**Prerequisites:** 
You need to have Vault up and running

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`

In [5]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [6]:
vault login root

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       dRq6usE3NFp7vjYtgtH8uqQv
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [ ]:
vault status; vault version

### Setup LDAP

In [7]:
docker stop ldap-service;
sleep 5;
docker run \
  --name ldap-service \
  --env LDAP_ORGANISATION="engineers" \
  --env LDAP_DOMAIN="example.org" \
  --env LDAP_ADMIN_PASSWORD="admin" \
  -p 389:389 \
  -p 636:636 \
  --detach \
  --rm \
  osixia/openldap:latest

Error response from daemon: No such container: ldap-service
c22e095f69d718d791ae324960616f8ef4b978745466c6940b6d32af77e1f4f6


In [8]:
docker ps -f name=ldap-service --format "table {{.Names}}\t{{.Status}}"

NAMES          STATUS
ldap-service   Up 8 seconds


### Adding entries to LDAP

In [10]:
cat > /tmp/ldap-example.ldif <<EOF
dn: ou=groups,dc=example,dc=org
objectClass: organizationalunit
objectClass: top
ou: groups
description: groups of users

dn: ou=users,dc=example,dc=org
objectClass: organizationalunit
objectClass: top
ou: users
description: users

dn: cn=dev,ou=groups,dc=example,dc=org
objectClass: groupofnames
objectClass: top
description: testing group for dev
cn: dev
member: cn=alice,ou=users,dc=example,dc=org
member: cn=bob,ou=users,dc=example,dc=org

dn: cn=manager,ou=groups,dc=example,dc=org
objectClass: groupofnames
objectClass: top
description: testing group for dev
cn: manager
member: cn=raymond,ou=users,dc=example,dc=org

dn: cn=alice,ou=users,dc=example,dc=org
objectClass: person
objectClass: top
cn: alice
sn: lee
memberOf: cn=dev,ou=groups,dc=example,dc=org
userPassword: passw0rd

dn: cn=bob,ou=users,dc=example,dc=org
objectClass: person
objectClass: top
cn: bob
sn: villa
memberOf: cn=dev,ou=groups,dc=example,dc=org
userPassword: passw0rd

dn: cn=raymond,ou=users,dc=example,dc=org
objectClass: person
objectClass: top
cn: raymond
sn: goh
memberOf: cn=manager,ou=groups,dc=example,dc=org
userPassword: passw0rd
EOF

In [11]:
#-c continous operation mode (do not stop on errors)
#-x simple authentication
#-W prompt for bind password
#-D bindn 
ldapadd -cxD "cn=admin,dc=example,dc=org" -f /tmp/ldap-example.ldif -w admin

adding new entry "ou=groups,dc=example,dc=org"

adding new entry "ou=users,dc=example,dc=org"

adding new entry "cn=dev,ou=groups,dc=example,dc=org"

adding new entry "cn=manager,ou=groups,dc=example,dc=org"

adding new entry "cn=alice,ou=users,dc=example,dc=org"

adding new entry "cn=bob,ou=users,dc=example,dc=org"

adding new entry "cn=raymond,ou=users,dc=example,dc=org"



**Search ldap (test)**

In [12]:
docker exec ldap-service ldapsearch -x \
  -H ldap://localhost -b dc=example,dc=org -D "cn=admin,dc=example,dc=org" -w admin

# extended LDIF
#
# LDAPv3
# base <dc=example,dc=org> with scope subtree
# filter: (objectclass=*)
# requesting: ALL
#

# example.org
dn: dc=example,dc=org
objectClass: top
objectClass: dcObject
objectClass: organization
o: engineers
dc: example

# groups, example.org
dn: ou=groups,dc=example,dc=org
objectClass: organizationalUnit
objectClass: top
ou: groups
description: groups of users

# users, example.org
dn: ou=users,dc=example,dc=org
objectClass: organizationalUnit
objectClass: top
ou: users
description: users

# dev, groups, example.org
dn: cn=dev,ou=groups,dc=example,dc=org
objectClass: groupOfNames
objectClass: top
description: testing group for dev
cn: dev
member: cn=alice,ou=users,dc=example,dc=org
member: cn=bob,ou=users,dc=example,dc=org

# manager, groups, example.org
dn: cn=manager,ou=groups,dc=example,dc=org
objectClass: groupOfNames
objectClass: top
description: testing group for dev
cn: manager
member: cn=raymond,ou=users,dc=example,dc=org

# alice, users, example.org


### Configure web Admin frontend to LDAP

In [17]:
# docker rm phpldapadmin-service

docker run -p 6443:443 --name phpldapadmin-service --hostname phpldapadmin-service \
    --link ldap-service:ldap-host \
    --env PHPLDAPADMIN_LDAP_HOSTS=ldap-host \
    --detach osixia/phpldapadmin:latest

e86cbb6ba4506248f07f3784de6021b1cdff94e67b241e668155337c51728d5c


**Information about the containers**

In [18]:
PHPLDAP_IP=$(docker inspect -f "{{ .NetworkSettings.IPAddress }}" phpldapadmin-service)
LDAP_IP=$(docker inspect -f "{{ .NetworkSettings.IPAddress }}" ldap-service)
echo "open browser to https://localhost:6443"
echo "phpweb IP is $PHPLDAP_IP; ldap IP is $LDAP_IP"
echo "Login DN: cn=admin,dc=example,dc=org"
echo "Password: admin"

open browser to https://localhost:6443
phpweb IP is 172.17.0.5; ldap IP is 172.17.0.4
Login DN: cn=admin,dc=example,dc=org
Password: admin


### Setup Vault to talk to LDAP

In [19]:
export VAULT_ADDR=http://127.0.0.1:8200
vault login root
vault auth enable ldap

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       dRq6usE3NFp7vjYtgtH8uqQv
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]
Success! Enabled ldap auth method at: ldap/


### Setup Vault to search LDAP

In [20]:
vault write auth/ldap/config \
    url=ldap://127.0.0.1 \
    userdn="ou=users,dc=example,dc=org" \
    groupdn="ou=groups,dc=example,dc=org" \
    binddn="cn=admin,dc=example,dc=org" \
    bindpass=admin \
    token_policies=dynamic-policy

Success! Data written to: auth/ldap/config


In [ ]:
VAULT_TOKEN=root vault read auth/ldap/config

### Login to Vault using LDAP credential

In [21]:
vault login -method=ldap username=raymond password=passw0rd

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                    Value
---                    -----
token                  s.BzZfT2G4DsLW9FIIWtAStoMg
token_accessor         wf7uvNwWd4AFjBb3FKdP63uv
token_duration         768h
token_renewable        true
token_policies         ["default" "dynamic-policy"]
identity_policies      []
policies               ["default" "dynamic-policy"]
token_meta_username    raymond


### Cleanup

In [ ]:
vault auth disable ldap

In [ ]:
docker stop vault-ldap

In [ ]:
docker rm vault-ldap

In [ ]:
docker stop phpldapadmin-service

In [ ]:
docker rm phpldapadmin-service

---
### View Vault via a browser
Open a web browser and visit http://127.0.0.1:8200/ui/vault
Use **TOKEN** and **Sign in**

### TODO:
- how to set AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS for a certain profile

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">